In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
def extractStockData(stockCode):
    
    response = requests.get("https://www.stocklore.ai/api/nse_data.csv?q="+stockCode)
    
    if(response.status_code == 200):
        
        a = response.content.decode("utf-8").split("\n")
        a = [i.split(",") for i in a]
        
        price_df = pd.DataFrame(a[1:-1], columns=a[0])
        price_df.iloc[:,1:] = price_df.iloc[:,1:].astype('float')
        price_df['Date'] = pd.to_datetime(price_df['Date'], format='%Y-%m-%d')
        
        return price_df
    
    else:
        print("Error in Stock Price Data Extraction, Check the stock code name and internet connection")    
        

In [ ]:
def scrapeSplitData()

In [ ]:
def adjust_split(stockCode):
    
    price_df = extractStockData(stockCode)
    
    